In [1]:
from sqlalchemy import create_engine

engine = create_engine('sqlite:///example.db', echo=True)

### ORM 초기화

In [2]:
from sqlalchemy.orm import sessionmaker
from sqlalchemy.ext.declarative import declarative_base

session_maker = sessionmaker(bind=engine)
session = session_maker()

Base = declarative_base()

/tmp/ipykernel_93940/2531115412.py:7: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


### 테이블 정의

In [3]:
from sqlalchemy import Column, Integer, String

class Demo(Base):
    __tablename__ = 'demos'
    
    id = Column(Integer, primary_key=True)
    name = Column(String)
    password = Column(String)

In [5]:
Demo.__tablename__

'demos'

### metadata확인
* demos 테이블은 engine이 메타데이터로 관리
* 아직 databse에 미반영

In [6]:
for k,v in Base.metadata.tables.items():
    print(f"{k}: {v}")

demos: demos


In [7]:
Base.metadata.tables

FacadeDict({'demos': Table('demos', MetaData(), Column('id', Integer(), table=<demos>, primary_key=True, nullable=False), Column('name', String(), table=<demos>), Column('password', String(), table=<demos>), schema=None)})

### metadata -> 데이터베이스 반영
* engine은 commit을 실행하여 데이터베이스에 테이블을 반영(실행 결과 맨 마지막 줄에 commit로그)

In [8]:
Base.metadata.create_all(engine)

2024-06-29 01:34:32,284 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-06-29 01:34:32,285 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("demos")
2024-06-29 01:34:32,286 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-06-29 01:34:32,291 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("demos")
2024-06-29 01:34:32,292 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-06-29 01:34:32,294 INFO sqlalchemy.engine.Engine 
CREATE TABLE demos (
	id INTEGER NOT NULL, 
	name VARCHAR, 
	password VARCHAR, 
	PRIMARY KEY (id)
)


2024-06-29 01:34:32,295 INFO sqlalchemy.engine.Engine [no key 0.00076s] ()
2024-06-29 01:34:32,314 INFO sqlalchemy.engine.Engine COMMIT


### 세션에 객체 추가
* 세션에만 객체가 저장될 뿐 아직 데이터베이스에 미반영

In [10]:
# 객체 생성
demo = Demo(name="hello", password="world")

session.add(demo)

In [11]:
# 생성한 객체 조회
find_demos = session.query(Demo).filter_by(name="hello").all()

for demo in find_demos:
    print(demo.name)

2024-06-29 01:35:12,530 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-06-29 01:35:12,535 INFO sqlalchemy.engine.Engine INSERT INTO demos (name, password) VALUES (?, ?)
2024-06-29 01:35:12,536 INFO sqlalchemy.engine.Engine [generated in 0.00131s] ('hello', 'world')
2024-06-29 01:35:12,545 INFO sqlalchemy.engine.Engine INSERT INTO demos (name, password) VALUES (?, ?)
2024-06-29 01:35:12,546 INFO sqlalchemy.engine.Engine [cached since 0.01165s ago] ('hello', 'world')
2024-06-29 01:35:12,549 INFO sqlalchemy.engine.Engine SELECT demos.id AS demos_id, demos.name AS demos_name, demos.password AS demos_password 
FROM demos 
WHERE demos.name = ?
2024-06-29 01:35:12,550 INFO sqlalchemy.engine.Engine [generated in 0.00103s] ('hello',)
hello
hello


### 세션 commit
* 세션에 저장된 데이터를 데이터베이스에 commit

In [12]:
session.commit()

2024-06-29 01:35:36,336 INFO sqlalchemy.engine.Engine COMMIT
